In [1]:
%load_ext autoreload
% autoreload 2

In [2]:
!pip install --upgrade pip

     |████████████████████████████████| 1.4MB 4.5MB/s eta 0:00:01
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [3]:
!yes | pip uninstall -q tensorflow
!pip install -q tensorflow==2.1.0rc2

Proceed (y/n)? yes: standard output: Broken pipe
yes: write error
ERROR: tensorflow-serving-api 1.15.0 has requirement tensorflow~=1.15.0, but you'll have tensorflow 2.1.0rc2 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.10.1 which is incompatible.


In [ ]:
# TODO - Is there an error in the loading that I need to fix?

In [4]:
import tensorflow as tf
tf.__version__

[autoreload of six failed: Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 384, in superreload
    update_generic(old_obj, new_obj)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 323, in update_generic
    update(a, b)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 275, in update_class
    old_obj = getattr(old, key)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/six.py", line 93, in __get__
    setattr(obj, self.name, result)  # Invokes __set__.
AttributeError: 'NoneType' object has no attribute 'cStringIO'
]


'2.1.0-rc2'

In [5]:
# !yes | pip uninstall tensorflow
# ! pip install tensorflow-gpu==2.0.0

In [18]:
tf.constant([2.0, 2.0])

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 2.], dtype=float32)>

In [19]:
import random
import math
import datetime
import csv
import re
import os
from random import shuffle
from collections import defaultdict	

In [43]:
minimum_evaluation_items = 1200 # annotate this many randomly sampled items first for evaluation data before creating training data
#minimum_training_items = 400 # minimum number of training items before we first train a model
minimum_training_items = 100 # minimum number of training items before we first train a model

epochs = 10 # number of epochs per training session
select_per_epoch = 200  # number to select per epoch per label


data = []
test_data = []

In [44]:
# directories with data
unlabeled_data = "unlabeled_data/unlabeled_data.csv"

evaluation_related_data = "evaluation_data/related.csv"
evaluation_not_related_data = "evaluation_data/not_related.csv"

#validation_related_data # not used in this example
#validation_not_related_data # not used in this example

training_related_data = "training_data/related.csv"
training_not_related_data = "training_data/not_related.csv"


already_labeled = {} # tracking what is already labeled
feature_index = {} # feature mapping for one-hot encoding

In [45]:
def load_data(filepath, skip_already_labeled=False):
    # csv format: [ID, TEXT, LABEL, SAMPLING_STRATEGY, CONFIDENCE]
    with open(filepath, 'r') as csvfile:
        data = []
        reader = csv.reader(csvfile)
        for row in reader:
            if skip_already_labeled and row[0] in already_labeled:
        	    continue
        		
            if len(row) < 3:
                row.append("") # add empty col for LABEL to add later
            if len(row) < 4:
                row.append("") # add empty col for SAMPLING_STRATEGY to add later        
            if len(row) < 5:
                row.append(0) # add empty col for CONFIDENCE to add later         
            data.append(row)

            label = str(row[2])
            if row[2] != "":
                textid = row[0]
                already_labeled[textid] = label

    csvfile.close()
    return data

In [46]:
def append_data(filepath, data):
    with open(filepath, 'a', errors='replace') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)
    csvfile.close()

def write_data(filepath, data):
    with open(filepath, 'w', errors='replace') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)
    csvfile.close()

In [47]:
training_data = load_data(training_related_data) + load_data(training_not_related_data)
training_count = len(training_data)
    
evaluation_data = load_data(evaluation_related_data) + load_data(evaluation_not_related_data)
evaluation_count = len(evaluation_data)

In [48]:
data = load_data(unlabeled_data, skip_already_labeled=True)

In [49]:
data[:4]

[['1001927',
  'wall st us shares drop aud rises despite interest rate cut',
  '',
  '',
  0],
 ['17349', 'israeli soldiers reportedly kill three palestinians', '', '', 0],
 ['91445', 'quarantine concerns over changed wheat import', '', '', 0],
 ['791309', 'diocese had fair share of paedophilia; inquiry told', '', '', 0]]

In [50]:
annotation_instructions = "Please type 1 if this message is disaster-related, "
annotation_instructions += "or hit Enter if not.\n"
annotation_instructions += "Type 2 to go back to the last message, "
annotation_instructions += "type d to see detailed definitions, "
annotation_instructions += "or type s to save your annotations.\n"

last_instruction = "All done!\n"
last_instruction += "Type 2 to go back to change any labels,\n"
last_instruction += "or Enter to save your annotations."

detailed_instructions = "A 'disaster-related' headline is any story about a disaster.\n"
detailed_instructions += "It includes:\n"
detailed_instructions += "  - human, animal and plant disasters.\n"
detailed_instructions += "  - the response to disasters (aid).\n"
detailed_instructions += "  - natural disasters and man-made ones like wars.\n"
detailed_instructions += "It does not include:\n"
detailed_instructions += "  - criminal acts and non-disaster-related police work\n"
detailed_instructions += "  - post-response activity like disaster-related memorials.\n\n"

In [52]:



def get_annotations(data, default_sampling_strategy="random"):
    """Prompts annotator for label from command line and adds annotations to data 
    
    Keyword arguments:
        data -- an list of unlabeled items where each item is 
                [ID, TEXT, LABEL, SAMPLING_STRATEGY, CONFIDENCE]
        default_sampling_strategy -- strategy to use for each item if not already specified
    """

    ind = 0
    while ind <= len(data):
        if ind < 0:
            ind = 0 # in case you've gone back before the first
        if ind < len(data):
            textid = data[ind][0]
            text = data[ind][1]
            label = data[ind][2]
            strategy =  data[ind][3]

            if textid in already_labeled:
                print("Skipping seen "+label)
                ind+=1
            else:
                print(annotation_instructions)
                label = str(input(text+"\n\n> ")) 

                if label == "2":                   
                    ind-=1  # go back
                elif label == "d":                    
                    print(detailed_instructions) # print detailed instructions
                elif label == "s":
                    break  # save and exit
                else:
                    if not label == "1":
                        label = "0" # treat everything other than 1 as 0
                        
                    data[ind][2] = label # add label to our data

                    if data[ind][3] is None or data[ind][3] == "":
                        data[ind][3] = default_sampling_strategy # add default if none given
                    ind+=1        

        else:
            #last one - give annotator a chance to go back
            print(last_instruction)
            label = str(input("\n\n> ")) 
            if label == "2":
                ind-=1
            else:
                ind+=1

    return data

In [53]:
# What exactly is the part that is using the 

# the length of the eval data 
# how many items need to be evaluated before creating training data


evaluation_count

1200

In [54]:
len(data)

1062013

In [55]:
# Text Classifier goes here

In [57]:
if evaluation_count <  minimum_evaluation_items:
    #Keep adding to evaluation data first
    print("Creating evaluation data:\n")

    shuffle(data)
    needed = minimum_evaluation_items - evaluation_count
    data = data[:needed]
    print(str(needed)+" more annotations needed")

    data = get_annotations(data) 
	
    related = []
    not_related = []

    for item in data:
        label = item[2]    
        if label == "1":
            related.append(item)
        elif label == "0":
            not_related.append(item)

    # append evaluation data
    append_data(evaluation_related_data, related)
    append_data(evaluation_not_related_data, not_related)
    
elif training_count < minimum_training_items:
    # lets create our first training data! 
    print("Creating initial training data:\n")

    shuffle(data)
    needed = minimum_training_items - training_count
    data = data[:needed]
    print(str(needed)+" more annotations needed")

    data = get_annotations(data)

    related = []
    not_related = []

    # RAB - Related is the stuff that is associated with disaster
    for item in data:
        label = item[2]
        if label == "1":
            related.append(item)
        elif label == "0":
            not_related.append(item)

    # append training data
    append_data(training_related_data, related)
    append_data(training_not_related_data, not_related)

Creating initial training data:

Please type 1 if this message is disaster-related, or hit Enter if not.
Type 2 to go back to the last message, type d to see detailed definitions, or type s to save your annotations.



community debates bega west school future

>  


Please type 1 if this message is disaster-related, or hit Enter if not.
Type 2 to go back to the last message, type d to see detailed definitions, or type s to save your annotations.



outrage over alves banana racist taunt

>  


Please type 1 if this message is disaster-related, or hit Enter if not.
Type 2 to go back to the last message, type d to see detailed definitions, or type s to save your annotations.



...

irish coup as mcdermott signs on

>  


Please type 1 if this message is disaster-related, or hit Enter if not.
Type 2 to go back to the last message, type d to see detailed definitions, or type s to save your annotations.



nsw premier kristina keneally and mark tobin sing

>  


Please type 1 if this message is disaster-related, or hit Enter if not.
Type 2 to go back to the last message, type d to see detailed definitions, or type s to save your annotations.



correctional services investigate prisoners death

>  s


In [58]:
related = []
not_related = []

In [59]:
data[:10]

[['457540', 'community debates bega west school future', '0', 'random', 0],
 ['853598', 'outrage over alves banana racist taunt', '0', 'random', 0],
 ['37513', '13 year old refused bail over sydney murder', '0', 'random', 0],
 ['790868',
  'gunnedah losing out as miners lure skilled labourer',
  '0',
  'random',
  0],
 ['544136',
  'horrendous sweatshops ditched for australian made',
  '0',
  'random',
  0],
 ['98625', 'new laws target prisoners mobile phone use', '0', 'random', 0],
 ['815439',
  'a league highlights melbourne victory v brisbane',
  '0',
  'random',
  0],
 ['847332', 'un announces sri lanka war crimes probe', '0', 'random', 0],
 ['252057', 'un yet to decide on auschwitz name change', '0', 'random', 0],
 ['148229', 'community disturbed by riot violence', '1', 'random', 0]]

In [60]:
import pdb

In [61]:
def create_features(minword = 3):
    """Create indexes for one-hot encoding of words in files
    
    """

    pdb.set_trace()
    total_training_words = {}
    for item in data + training_data:
        text = item[1]
        for word in text.split():
            if word not in total_training_words:
                total_training_words[word] = 1
            else:
                total_training_words[word] += 1

    for item in data + training_data:
        text = item[1]
        for word in text.split():
            if word not in feature_index and total_training_words[word] >= minword:
                feature_index[word] = len(feature_index)

    return len(feature_index)

In [ ]:

# At this point I am assuming that I have enough training data and enough evaluation data
vocab_size = create_features()

> <ipython-input-61-1a2da448de1a>(7)create_features()
-> total_training_words = {}


(Pdb)  n


> <ipython-input-61-1a2da448de1a>(8)create_features()
-> for item in data + training_data:


(Pdb)  len(data)


1062013


...

> <ipython-input-61-1a2da448de1a>(12)create_features()
-> total_training_words[word] = 1


(Pdb)  n


> <ipython-input-61-1a2da448de1a>(10)create_features()
-> for word in text.split():


(Pdb)  total_training_words


{'community': 1}
